In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=4546ff5b52f874fe36258b14f43711e304eefab004e9957bf253fec54bdbdec8
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F16 '''
def F16(X):
    f = bn.F16()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.0667 ,random_state=0 , l1_ratio=0.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 10.83 , C = 0.00010 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_16_4000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_16_800Samples.csv').iloc[:,-1]
path = "train_latent_200D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_200D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 85707.49452089125, tolerance: 18.91816706965524
  positive)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=912806, Mon Apr 13 18:01:24 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 -1.801812808110249e+02 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 18:01:24 2020)
final/bestever f-value = -1.801813e+02 -1.801813e+02
incumbent solution: [0.80841385 1.00559547 0.27430499 0.6728318  0.40237109 0.59188193
 0.02539921 0.47585938 ...]
std deviations: [0.15023047 0.15024383 0.15024844 0.15024934 0.15017461 0.15031915
 0.15029387 0.15028415 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=983475, Mon Apr 13 18:01:24 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 -1.804066352716102e+02 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.18128081] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-180.181281, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-180.181281, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

termination on tolfun=1e-11 (Mon Apr 13 18:01:24 2020)
final/bestever f-value = -1.804066e+02 -1.804066e+02
incumbent solution: [0.8719856  0.93252448 0.42618096 0.52186089 0.4085545  0.55667432
 0.0579511  0.30098832 ...]
std deviations: [0.15056027 0.15054519 0.15052867 0.15053878 0.15050806 0.15051143
 0.15046031 0.15054071 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=932360, Mon Apr 13 18:01:24 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 -1.822821599586156e+02 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
    2     34 -1.826789621258731e+02 1.0e+00 1.46e-01  1e-01  1e-01 0:00.1
    3     51 -1.840160406725435e+02 1.0e+00 1.43e-01  1e-01  1e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.1716119] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-181.55836407] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   80   1360 -4.128157440209813e+03 1.3e+00 2.03e+00  2e+00  2e+00 0:03.1
  100   1700 -2.190052075342627e+04 1.3e+00 5.29e+00  5e+00  5e+00 0:03.9
  172   2924 -1.716020882729790e+07 1.3e+00 1.56e+02  2e+02  2e+02 0:06.8
termination on tolfacupx=1000.0 (Mon Apr 13 18:01:32 2020)
final/bestever f-value = -1.637125e+07 -1.716021e+07
incumbent solution: [ 226.13968908 -170.42421064  620.32550962   66.48015773  248.52897505
 -281.51295756   66.92938591 -978.79578294 ...]
std deviations: [153.9932426  153.59872    154.16702505 154.36741664 153.7700589
 154.75299057 154.22304651 154.74171491 ...]
Run : 1
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=975832, Mon Apr 13 18:01:32 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 -1.801812808110249e+02 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 18:01:32 2020)
final/bestever f-value = -1.801813e+02 -1.801813e+02
incumbent solution: [0.13681937 0.35339612 0.6

...

   81   1377 -5.489662760152900e+03 1.3e+00 2.68e+00  3e+00  3e+00 0:03.1
  100   1700 -3.646032473867415e+04 1.3e+00 7.06e+00  7e+00  7e+00 0:03.9
  161   2737 -1.567138635802297e+07 1.3e+00 1.60e+02  2e+02  2e+02 0:06.3
termination on tolfacupx=1000.0 (Mon Apr 13 18:05:15 2020)
final/bestever f-value = -1.531929e+07 -1.567139e+07
incumbent solution: [-620.39676968 1218.33546814  659.72594245 -155.04858776  206.1854272
   92.02733832  385.17577638 -498.70203476 ...]
std deviations: [159.50854595 160.18103378 158.92196797 156.84427111 158.34024299
 158.46700519 157.87464493 158.67214042 ...]
Run : 29
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=951416, Mon Apr 13 18:05:15 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 -1.801812808110249e+02 1.0e+00 1.50e-01  1e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 18:05:15 2020)
final/bestever f-value = -1.801813e+02 -1.801813e+02
incumbent solution: [0.1316263  0.80053157 0.

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.18128081] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-180.181281, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-180.181281, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

termination on tolfun=1e-11 (Mon Apr 13 18:05:15 2020)
final/bestever f-value = -1.804066e+02 -1.804066e+02
incumbent solution: [-0.00946405  0.80532066  0.81853149  0.93992494  0.31249692  0.12408424
  0.47758132  0.65034179 ...]
std deviations: [0.15039423 0.15031901 0.15031579 0.15034662 0.1503782  0.15041456
 0.1503422  0.15036358 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=968615, Mon Apr 13 18:05:15 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 -1.797681134772591e+02 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
    2     34 -1.813050818744666e+02 1.0e+00 1.45e-01  1e-01  1e-01 0:00.1
    3     51 -1.822871460767074e+02 1.0e+00 1.42e-01  1e-01  1e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-178.39537862] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.55477176] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   80   1360 -4.307714330247135e+03 1.3e+00 2.24e+00  2e+00  2e+00 0:03.1
  100   1700 -2.689416743033021e+04 1.3e+00 5.35e+00  5e+00  5e+00 0:03.9
  169   2873 -1.942042566768959e+07 1.4e+00 1.56e+02  2e+02  2e+02 0:06.6
termination on tolfacupx=1000.0 (Mon Apr 13 18:05:23 2020)
final/bestever f-value = -1.830527e+07 -1.942043e+07
incumbent solution: [-497.56074264   -6.61102248  819.43236072 -211.71175212   50.11101498
  530.85599189 -263.05674658 -446.87991435 ...]
std deviations: [154.42056837 153.52729663 155.01775994 153.47036666 153.62187855
 155.34679634 154.89826004 154.28449918 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F16_200_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F16(X_Values.iloc[i,:100])
    SVM_Fun [i] = F16(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F16(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([-208.60005138]), count=array([1])) 9.66182862427273
SVM
ModeResult(mode=array([-207.32753596]), count=array([1])) 7.1391978978273025
ELN
ModeResult(mode=array([2196934.16475031]), count=array([1])) 240661.37714146904
